In [ ]:
# %pip install python-dotenv
# %pip install pandas
# %pip install requests
# %pip install pyshacl
# %pip install brickschema

In [2]:
import requests
import pandas as pd
import json
from io import StringIO

# Developing artifacts for building 1

In [18]:
building1_sparql_endpoint = 'http://localhost:7400/repositories/building1'

### Read the artifact graph and list the required inputs for later validation

In [19]:
sparql_query = """
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX inst: <http://example.org/demo#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX schema: <https://schema.org/>
PREFIX sba: <https://example.org/sba-schema#>
PREFIX mpc: <https://example.org/sba-mpc-schema#>
PREFIX forc: <https://example.org/sba-forecast-schema#>

SELECT ?input ?datapoint ?label ?tsid ?database where { 
    ?algorithm a forc:LoadForecastAlgorithm .
	?algorithm ssn:hasInput ?input .

    OPTIONAL{
    ?input  ref:hasExternalReference ?arr .
            ?arr ref:hasTimeseriesId ?tsid .
            ?arr ref:storedAt ?db .
            ?db inst:connstring ?database .
            ?input a ?datapoint .
        OPTIONAL {
        	?input rdfs:label ?label .
    	}
    }
}
"""

query_params = {
    'query': sparql_query,
    'infer': 'false' 
}

response = requests.get(building1_sparql_endpoint, params=query_params)

if response.status_code == 200:
    text = pd.read_csv(StringIO(response.text), sep=",")
    df = pd.DataFrame(text, columns=["input", "datapoint", "label" ,"tsid", "database"], index=None)
else:
    print(f"Error: {response.status_code} - {response.reason}")
df

,input,datapoint,label,tsid,database
0,http://example.org/demo#bldg_virtual_power,https://brickschema.org/schema/Brick#Electric_...,Building Load (kW),building.Value,https://ivs.kropman.nl/insitereports/
1,http://example.org/demo#bldgOutdoorTemperature...,https://example.org/sba-schema#OutdoorTemperat...,OutdoorTemperatureForecast,3ede3817d96d7bb87865eaa7ac657860.Value,https://ivs.kropman.nl/insitereports/
2,http://example.org/demo#bldgSolarRadianceForecast,https://example.org/sba-schema#SolarRadianceFo...,SolarRadianceForecast,3ede3817d96d7bb87865eaa7ac657860.Value,https://ivs.kropman.nl/insitereports/
3,http://example.org/demo#bldgWindSpeedForecast,https://example.org/sba-schema#WindSpeedForecast,WindSpeedForecast,3ede3817d96d7bb87865eaa7ac657860.Value,https://ivs.kropman.nl/insitereports/
4,http://example.org/demo#bldgTrainedRFModel,https://example.org/sba-forecast-schema#Buildi...,TrainedRFModel,model1.sav,https://bimsim.app/mpc
5,http://example.org/demo#bldgForecastModelParam...,https://example.org/sba-forecast-schema#Buildi...,ForecastModelParameters,modelparamas.json,https://bimsim.app/mpc
6,http://example.org/demo#bldgOutdoorTemperature...,https://brickschema.org/schema/Brick#Outside_A...,TEMP,a683b494a806af9a8da4d35ae99ba763.Value,https://ivs.kropman.nl/insitereports/
7,http://example.org/demo#bldgWindSpeedSensor,https://brickschema.org/schema/Brick#Wind_Spee...,WIND,a683b494a806af9a8da4d35ae99ba763.Value,https://ivs.kropman.nl/insitereports/
8,http://example.org/demo#bldgSolarRadianceSensor,https://brickschema.org/schema/Brick#Solar_Rad...,GHI,a683b494a806af9a8da4d35ae99ba763.Value,https://ivs.kropman.nl/insitereports/


In [20]:
dplist = df['datapoint'].tolist()
dplist

['https://brickschema.org/schema/Brick#Electric_Power_Sensor',
 'https://example.org/sba-schema#OutdoorTemperatureForecast',
 'https://example.org/sba-schema#SolarRadianceForecast',
 'https://example.org/sba-schema#WindSpeedForecast',
 'https://example.org/sba-forecast-schema#BuildingLoadForecastTrainedModel',
 'https://example.org/sba-forecast-schema#BuildingLoadForecastModelParameters',
 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor',
 'https://brickschema.org/schema/Brick#Wind_Speed_Sensor',
 'https://brickschema.org/schema/Brick#Solar_Radiance_Sensor']

In [21]:
point_with_suffix_list = []
for point in dplist:
    suffix = point.split('#')[0] + '#'
    point = point.split('#')[-1]
    prefixes = {
        'brick': 'https://brickschema.org/schema/Brick#',
        'sba': 'https://example.org/sba-schema#',
        'forc': 'https://example.org/sba-forecast-schema#',
        'mpc': 'https://example.org/sba-mpc-schema#',
        
    }
    for prefix, uri in prefixes.items():
        if uri == suffix:
            suffix = prefix
            point_with_suffix = f'{suffix}:{point}'
            point_with_suffix_list.append(point_with_suffix)
            break
point_with_suffix_list

['brick:Electric_Power_Sensor',
 'sba:OutdoorTemperatureForecast',
 'sba:SolarRadianceForecast',
 'sba:WindSpeedForecast',
 'forc:BuildingLoadForecastTrainedModel',
 'forc:BuildingLoadForecastModelParameters',
 'brick:Outside_Air_Temperature_Sensor',
 'brick:Wind_Speed_Sensor',
 'brick:Solar_Radiance_Sensor']

### Creating and saving all the shape graphs based on Algorithm Inputs

In [23]:
shape_graph_list = []
class_names = []
for point in point_with_suffix_list:
    shape_graph = f"""
@prefix inst: <http://example.org/demo#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#>.
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix sba: <https://example.org/sba-schema#> .
@prefix eva: <https://example.org/sba-ev-schema#> .
@prefix mpc: <https://example.org/sba-mpc-schema#> .
@prefix forc: <https://example.org/sba-forecast-schema#> .

inst:AlgorithmInputShape
    a sh:NodeShape ;
    sh:targetClass {point} ;
    sh:property [
        sh:path ref:hasExternalReference ;
        sh:minCount 1 ;
        sh:node sh:NodeShape ;
        sh:property [
            sh:path rdf:type ;
            sh:hasValue ref:TimeseriesReference ;
        ] ;
        sh:property [
            sh:path ref:hasTimeseriesId ;
            sh:minCount 1 ;
            sh:datatype xsd:string ;
        ] ;
    ] .

"""
    shape_graph_list.append(shape_graph)
    class_name = point.replace(":", "_")
    class_names.append(class_name)

i=0
for g in shape_graph_list:
    with open(f"ttl/shape_graphs/shape_graph_{class_names[i]}.ttl", "w") as f:
        f.write(g)
        i=i+1

#### Modify shape_graph_brick_Electric_Power_Sensor.ttl to restrict the Electric_Power_Sensor to be a PointOf brick:Building_Electric_Meter (because we are interested in the whole building load)

In [24]:
from rdflib import Graph, Namespace, RDF, URIRef, BNode, Literal
from rdflib.namespace import SH, XSD

BRICK = Namespace("https://brickschema.org/schema/Brick#")
INST = Namespace("http://example.org/demo#")

g = Graph()
g.parse('ttl/shape_graphs/shape_graph_brick_Electric_Power_Sensor.ttl', format='turtle')

property_shape = BNode()
g.add((property_shape, RDF.type, SH.PropertyShape))
g.add((property_shape, SH.path, BRICK.isPointOf))
g.add((property_shape, SH['class'], BRICK.Building_Electrical_Meter))
g.add((property_shape, SH.minCount, Literal(1, datatype=XSD.integer)))

shape_node = INST.AlgorithmInputShape
g.add((shape_node, SH.property, property_shape))

print(g.serialize(format='turtle'))

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix inst: <http://example.org/demo#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

inst:AlgorithmInputShape a sh:NodeShape ;
    sh:property [ a sh:PropertyShape ;
            sh:class brick:Building_Electrical_Meter ;
            sh:minCount 1 ;
            sh:path brick:isPointOf ],
        [ sh:minCount 1 ;
            sh:node sh:NodeShape ;
            sh:path ref:hasExternalReference ;
            sh:property [ sh:hasValue ref:TimeseriesReference ;
                    sh:path rdf:type ],
                [ sh:datatype xsd:string ;
                    sh:minCount 1 ;
                    sh:path ref:hasTimeseriesId ] ] ;
    sh:targetClass brick:Electric_Power_Sensor .




In [25]:
# replace shape_graph_brick_Electric_Power_Sensor.ttl with new shape graph
with open('ttl/shape_graphs/shape_graph_brick_Electric_Power_Sensor.ttl', 'w', encoding='utf-8' ) as f:
    f.write(g.serialize(format='turtle'))

### Merge all shape graphs to create one shape graph

In [10]:
from rdflib import Graph
import os

merged_graph = Graph()
folder_path = "ttl\shape_graphs"
for file_name in os.listdir(folder_path):
    if file_name.endswith('.ttl'):
        file_path = os.path.join(folder_path, file_name)
        temp_graph = Graph()
        temp_graph.parse(file_path, format='turtle')
        merged_graph += temp_graph
merged_graph.serialize("ttl/shape_graphs/merged_shape_graphs.ttl", format='turtle')

<Graph identifier=N633085d9d3b945b8822198b09e17650b (<class 'rdflib.graph.Graph'>)>

# Use shape graphs to validate building 2

In [3]:
building2_sparql_endpoint = 'http://localhost:7400/repositories/building2'

### Building 2 data graph

In [5]:
from rdflib import Graph

construct_query = """
CONSTRUCT { ?s ?p ?o }
WHERE { ?s ?p ?o }
"""

query_params = {
    'query': construct_query,
    'infer': 'false' 
}

response = requests.get(building2_sparql_endpoint, params=query_params)

if response.status_code == 200:
    response_text = response.text
    building2_data_graph= Graph()
    building2_data_graph.parse(data=response_text, format="turtle")
    print(f"Loaded {len(building2_data_graph)} tripples")

else:
    print(f"Error: {response.status_code} - {response.reason}")

Loaded 602 tripples


#### ASK Query

In [6]:
arr = ['brick:Electric_Power_Sensor',
 'sba:OutdoorTemperatureForecast',
 'sba:SolarRadianceForecast',
 'sba:WindSpeedForecast',
 'forc:BuildingLoadForecastTrainedModel',
 'forc:BuildingLoadForecastModelParameters',
 'brick:Outside_Air_Temperature_Sensor',
 'brick:Wind_Speed_Sensor',
 'brick:Solar_Radiance_Sensor']

In [7]:
ask_arr = []
for point in arr:
        ask_query = f"""
        PREFIX ssn: <http://www.w3.org/ns/ssn/>
        PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
        PREFIX inst: <http://example.org/demo#>
        PREFIX brick: <https://brickschema.org/schema/Brick#>
        PREFIX schema: <https://schema.org/>
        PREFIX sba: <https://example.org/sba-schema#>
        PREFIX mpc: <https://example.org/sba-mpc-schema#>
        PREFIX forc: <https://example.org/sba-forecast-schema#>

        ASK WHERE {{
            ?s a {point}
        }}
        """
        res = building2_data_graph.query(ask_query)
        boolean_result = bool(res.askAnswer)
        ask_arr.append((point, boolean_result))

for point, result in ask_arr:
    print(f"{point}: {result}")

brick:Electric_Power_Sensor: True
sba:OutdoorTemperatureForecast: True
sba:SolarRadianceForecast: True
sba:WindSpeedForecast: True
forc:BuildingLoadForecastTrainedModel: True
forc:BuildingLoadForecastModelParameters: True
brick:Outside_Air_Temperature_Sensor: True
brick:Wind_Speed_Sensor: True
brick:Solar_Radiance_Sensor: True


In [8]:
import pyshacl

def validate_graph(data_graph, shape_graph):
    shapes_file_format = "turtle"
    data_file_format = "turtle"

    res = pyshacl.validate(
        data_graph,
        shacl_graph=shape_graph,
        data_graph_format=data_file_format,
        shacl_graph_format=shapes_file_format,
        inference="both",
    )
    return res

In [11]:
res=validate_graph(merged_graph,building2_data_graph )
conforms, results_graph, results_text = res
print(results_text)

Validation Report
Conforms: True

